### Translation Task:

La traduction est une tâche de type séquence-à-séquence, similaire au résumé de texte. Elle peut également être adaptée à d'autres problèmes de ce genre, comme le transfert de style (par exemple, traduire un texte formel en un texte plus décontracté) ou la génération de réponses à des questions en fonction d'un contexte.

Si vous disposez d'un grand corpus de textes dans deux langues ou plus, vous pouvez entraîner un nouveau modèle de traduction à partir de zéro. Toutefois, il est souvent plus rapide de faire un fine-tuning d’un modèle de traduction existant, comme un modèle multilingue tel que mT5 ou mBART, ou un modèle spécialisé pour la traduction d’une langue à une autre.

Dans cette section, un modèle Marian pré-entraîné pour la traduction de l’anglais vers le français sera ajusté (fine-tuned) en utilisant le jeu de données KDE4. Ce modèle a été initialement entraîné sur un large corpus de textes en anglais et en français, et nous allons améliorer ses performances après l’étape de fine-tuning.

Une fois l’entraînement terminé, le modèle pourra faire des prédictions de traduction.

#### 1- Preparing the data


In [20]:
# Import librairies
import transformers
from datasets import load_dataset
from transformers import pipeline

In [11]:
# Lire our dataset
dataset = load_dataset("kde4", lang1 = "en", lang2 = "fr")
dataset

c:\Users\Zakaria-Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

Nous avons 210 173 paires de phrases, mais elles sont regroupées en un seul ensemble, ce qui signifie que nous devons créer notre propre ensemble de validation. Un objet Dataset possède une méthode train_test_split() qui peut nous aider. Nous allons fournir une graine (seed) pour garantir la reproductibilité.

In [14]:
# Splitter our data in train and test
split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)

#Rename our test to "validation"
split_datasets["validation"] = split_datasets.pop('test')

split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [18]:
split_datasets["train"][10]["translation"]

{'en': 'Text Cursor Movement', 'fr': 'Mouvements du curseur de texte'}

Nous obtenons un dictionnaire contenant deux phrases dans les langues demandées. Une particularité de ce jeu de données, qui contient beaucoup de termes techniques en informatique, est que tous ces termes sont entièrement traduits en français. Cependant, les ingénieurs français laissent souvent les mots spécifiques à l'informatique en anglais lorsqu'ils parlent. Par exemple, le mot « threads » pourrait apparaître tel quel dans une phrase française, surtout dans une conversation technique, mais dans ce jeu de données, il est traduit par l'expression plus correcte « fils de discussion ». Le modèle pré-entraîné que nous utilisons, qui a été formé sur un corpus plus large de phrases en français et en anglais, choisit souvent l'option plus simple en laissant le mot tel quel.

In [26]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.